## Pydantic Output parser

In [1]:
from dotenv import load_dotenv
import os

env_path = "../OllamaSetup/.env"  # Relative path from PromptTemplates
load_dotenv(env_path)

# Check if variables are loaded
print(os.getenv("LANGCHAIN_ENDPOINT")) 

https://api.smith.langchain.com


In [2]:
from langchain_ollama import ChatOllama
base_url="http://localhost:11434"
#model = 'llama3.2:1b'
# model='sheldon'
model = 'llama3.2:3b'


llm = ChatOllama(
    base_url=base_url,
    model = model)

In [3]:
from langchain_core.prompts import (
                                        SystemMessagePromptTemplate,
                                        HumanMessagePromptTemplate,
                                        ChatPromptTemplate,
                                        PromptTemplate
)


In [4]:
from typing import Optional
from pydantic import BaseModel , Field
from langchain_core.output_parsers import PydanticOutputParser


In [13]:
class Joke(BaseModel):
    """" Joke to tell user  """

    setup : str = Field(description="The setup of the joke")
    punchline : str = Field(description="The punchline of the joke")
    rating : Optional[int] = Field(description="Rating of the joke is from 1 to 10 ",default=None)
    

In [6]:
parser= PydanticOutputParser(pydantic_object=Joke)
parser

PydanticOutputParser(pydantic_object=<class '__main__.Joke'>)

In [7]:
instruction = parser.get_format_instructions()
print(instruction)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"description": "\" Joke to tell user  ", "properties": {"setup": {"description": "The setup of the joke", "title": "Setup", "type": "string"}, "punchline": {"description": "The punchline of the joke", "title": "Punchline", "type": "string"}, "rating": {"anyOf": [{"type": "integer"}, {"type": "null"}], "description": "Rating of the joke is from 1 to 10", "title": "Rating"}}, "required": ["setup", "punchline", "rating"]}
```


In [8]:
Joke_prompt= PromptTemplate(
    template = ''' 
    Answer the user query with a joke. Here is your formating instructions .
    {format_instruction}

    Query: {query}
    Answer:
 ''',
 input_variables=['query'],
 partial_variables={'format_instruction': parser.get_format_instructions()}
)

Joke_prompt

PromptTemplate(input_variables=['query'], input_types={}, partial_variables={'format_instruction': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"description": "\\" Joke to tell user  ", "properties": {"setup": {"description": "The setup of the joke", "title": "Setup", "type": "string"}, "punchline": {"description": "The punchline of the joke", "title": "Punchline", "type": "string"}, "rating": {"anyOf": [{"type": "integer"}, {"type": "null"}], "description": "Rating of the joke is from 1 to 10", "title": "Rating"}}, "required": ["setup", "punchline", "rating"]}\n```'}, template=

In [9]:
Joke_Chain = Joke_prompt | llm
Joke_output = Joke_Chain.invoke({'query': 'Tell me a joke about the dog'})
print(Joke_output.content)

{
  "properties": {
    "setup": {"description": "Why did the dog go to the vet?", "title": "Setup", "type": "string"},
    "punchline": {"description": "Because he was feeling ruff!", "title": "Punchline", "type": "string"},
    "rating": {"anyOf": [{"type": "integer"}, {"type": "null"}], "description": "Rating of the joke is from 1 to 10", "title": "Rating"}
  },
  "required": ["setup", "punchline", "rating"]
}


### Joke with better format

In [10]:
Joke_Chain_mod = Joke_prompt | llm | parser
Joke_output_mod = Joke_Chain_mod.invoke({'query': 'Tell me a joke about the dog'})
print(Joke_output_mod)

setup='Why did the dog go to the vet?' punchline='Because he was feeling ruff!' rating=8


### parsing with .with_structured_output() method

In [12]:
output = llm.invoke('Tell me a joke about the dog')
print(output.content)

Why did the dog go to the vet?

Because he was feeling ruff!


In [14]:
structured_llm = llm.with_structured_output(Joke)
structured_output = structured_llm.invoke('Tell me a joke about the dog')
print(structured_output)

setup='Why did the dog go to the vet?' punchline='Because he was feeling ruff!' rating=None


## JSON Output Parser

In [15]:
from langchain_core.output_parsers import JsonOutputParser

In [16]:
parser= JsonOutputParser(pydantic_object=Joke)
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"description": "\" Joke to tell user  ", "properties": {"setup": {"description": "The setup of the joke", "title": "Setup", "type": "string"}, "punchline": {"description": "The punchline of the joke", "title": "Punchline", "type": "string"}, "rating": {"anyOf": [{"type": "integer"}, {"type": "null"}], "default": null, "description": "Rating of the joke is from 1 to 10 ", "title": "Rating"}}, "required": ["setup", "punchline"]}
```


In [17]:
Joke_prompt= PromptTemplate(
    template = ''' 
    Answer the user query with a joke. Here is your formating instructions .
    {format_instruction}

    Query: {query}
    Answer:
 ''',
 input_variables=['query'],
 partial_variables={'format_instruction': parser.get_format_instructions()}
)

Joke_prompt

PromptTemplate(input_variables=['query'], input_types={}, partial_variables={'format_instruction': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"description": "\\" Joke to tell user  ", "properties": {"setup": {"description": "The setup of the joke", "title": "Setup", "type": "string"}, "punchline": {"description": "The punchline of the joke", "title": "Punchline", "type": "string"}, "rating": {"anyOf": [{"type": "integer"}, {"type": "null"}], "default": null, "description": "Rating of the joke is from 1 to 10 ", "title": "Rating"}}, "required": ["setup", "punchline"]}\n```'}, t

In [21]:
Json_Chain= Joke_prompt | llm
Json_output = Json_Chain.invoke({'query': 'Tell me a joke about the dog'})
print(Json_output.content)

{
  "properties": {
    "foo": {
      "description": "The punchline of the joke",
      "title": "Why did the dog go to the vet?",
      "type": "string"
    }
  },
  "required": ["foo"]
}


In [23]:
Json_Chain_mod= Joke_prompt | llm | parser
Json_output_mod=  Json_Chain_mod.invoke({'query': 'Tell me a joke about the dog'})
print(Json_output_mod)

{'setup': 'Why did the dog go to the vet?', 'punchline': 'Because he was feeling ruff!', 'rating': None}


## CSV Output Parser

In [37]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser

In [38]:
#parser= CommaSeparatedListOutputParser(pydantic_object=Joke)
parser= CommaSeparatedListOutputParser()
print(parser.get_format_instructions())

Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [39]:
format_instruction= parser.get_format_instructions

In [41]:
CSV_prompt= PromptTemplate(
    template = ''' 
    Answer the user query with a list of values. Here is your formating instructions .
    {format_instruction}

    Query: {query}
    Answer:
 ''',
 input_variables=['query'],
 partial_variables={'format_instruction': parser.get_format_instructions()}
)

CSV_prompt

PromptTemplate(input_variables=['query'], input_types={}, partial_variables={'format_instruction': 'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'}, template=' \n    Answer the user query with a list of values. Here is your formating instructions .\n    {format_instruction}\n\n    Query: {query}\n    Answer:\n ')

In [42]:
CSV_Chain= CSV_prompt | llm
CSV_output = CSV_Chain.invoke({'query': 'Generate my website seo keywords.I have content about NLP and LLM'})
print(CSV_output.content)

natural language processing, language models, artificial intelligence, machine learning, deep learning, sentiment analysis, text classification, natural language generation


In [43]:
CSV_Chain_mod= CSV_prompt | llm | parser
CSV_output_mod = CSV_Chain_mod.invoke({'query': 'Generate my website seo keywords.I have content about NLP and LLM'})
print(CSV_output_mod)

['nlp', 'llm', 'ai', 'machine learning', 'deep learning', 'natural language processing', 'language models', 'text analysis', 'semantic search', 'content generation', 'artificial intelligence', 'sentiment analysis', 'language understanding', 'chatbots', 'human-computer interaction', 'data science', 'machine learning applications.']


## Datetime Output Parsing

In [44]:
from langchain.output_parsers import DatetimeOutputParser

In [45]:
parser=DatetimeOutputParser()
format_instruction= parser.get_format_instructions
print(format_instruction)

<bound method DatetimeOutputParser.get_format_instructions of DatetimeOutputParser()>


In [46]:
Datetime_prompt= PromptTemplate(
    template = ''' 
    Answer the user query with a datetime. Here is your formating instructions .
    {format_instruction}

    Query: {query}
    Answer:
 ''',
 input_variables=['query'],
 partial_variables={'format_instruction': parser.get_format_instructions()}
)

Datetime_prompt

PromptTemplate(input_variables=['query'], input_types={}, partial_variables={'format_instruction': "Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.\n\nExamples: 1697-09-12T09:31:18.323380Z, 1687-07-28T06:09:35.826897Z, 1752-08-06T10:00:08.500496Z\n\nReturn ONLY this string, no other words!"}, template=' \n    Answer the user query with a datetime. Here is your formating instructions .\n    {format_instruction}\n\n    Query: {query}\n    Answer:\n ')

In [47]:
Datetime_chain= Datetime_prompt | llm | parser
Datetime_output = Datetime_chain.invoke({'query': 'When was America Discovered'})
print(Datetime_output)

1492-08-03 04:00:00


In [48]:
Datetime_chain= Datetime_prompt | llm | parser
Datetime_output = Datetime_chain.invoke({'query': 'When India got independence'})
print(Datetime_output)

1947-08-15 18:45:00
